In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# from pycaret.regression import *

df_train = pd.read_csv('../input/dont-overfit-ii/train.csv')
df_test = pd.read_csv('../input/dont-overfit-ii/test.csv')
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

**We can notice that the train and test data has different quantile range so they may need to scale all of them in the same quantile range using robust scaler**

In [ ]:
df_train.info()

In [ ]:
df_train.duplicated().sum()

In [ ]:
# null_col = df_train.isnull().sum()
df_train.columns[df_train.isna().any()].tolist()

In [ ]:
null_sum = df_train.isna().sum()
who_nulls = null_sum[null_sum > 0]
who_nulls

**From pervious inspections we can notice that the data already clean from duplicated and null values**

### 1) applying RobustScaler

In [ ]:
#applying robust scaler on all data will be better to make all data similer or in same scale because 
#train data is already small in compare with test data
from sklearn.preprocessing import RobustScaler
x = df_train.drop(['id','target'], axis=1).values
y = df_train['target']
df_test = df_test.drop(['id'], axis=1).values
all_data = RobustScaler().fit_transform(np.concatenate((x, df_test), axis=0))
x = all_data[:250]
test = all_data[250:]

In [ ]:
x = pd.DataFrame(x)
y = pd.DataFrame(y)
train_data = pd.concat([x,y],axis = 1)
train_data.head()

In [ ]:
train_data.info()

**In the next cell we can apply pycart library to give us indication about best model that give reasonable acuurcy with our data** 

In [ ]:
# !pip install pycaret
# from pycaret.classification import *

In [ ]:
# exp_clf = setup(train_data, target = 'target')

In [ ]:
# # compare_models(exclude = ['catboost', 'lightgbm'])
# compare_models()

**We can notice that the logistic legression model give high accurcy with AUC validation test so we will work with these model and apply hyper paramter tunning until reach to model that not overfitting or underfitting**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x, y,test_size=0.20,random_state = 0)

In [ ]:
#from sklearn.svm import SVC
#classifier=SVC(kernel='linear',class_weight='balanced',gamma='auto',probability=True)#This one overfit BEAUTIFULLY
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression(class_weight='balanced',penalty='l1',C=0.1,random_state=0,solver='liblinear')#L1='Lasso',l2="Ridge"
classifier.fit(X_train,y_train)
#print(classifier.score(X_train,y_train))

In [ ]:
def auc_curve():
    from sklearn import metrics
    y_pred_proba=classifier.predict_proba(X_val)[:,1]
    fpr,tpr,_=metrics.roc_curve(y_val,y_pred_proba)
    auc=metrics.roc_auc_score(y_val,y_pred_proba)
    plt.plot(fpr,tpr,label="auc="+str(auc))
    plt.legend(loc=4)
    plt.show()
    return y_pred_proba
y_pred_proba=auc_curve()

In [ ]:
y_pred=classifier.predict(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_val,y_pred)
cm

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies=cross_val_score(classifier,X_val,y_val,cv=10)
print(accuracies.mean())

In [ ]:
y_final_pred=classifier.predict(test)
sub=pd.read_csv("../input/dont-overfit-ii/sample_submission.csv")
submission_df = pd.DataFrame()
submission_df["ID"] = sub.id

submission_df["target"] = y_final_pred

In [ ]:
# submission_df.head(20)

In [ ]:
submission_df.to_csv("submission1.csv", index = False, header = True)

In [ ]:
# model = create_model('et')
# tuned_model = tune_model('lasso')
# predictions = predict_model(tuned_model)